In [252]:
import pdfplumber
import pandas as pd
import numpy as np

In [253]:
# define the columns as a list
# the two pages of the table are parsing with different number of columns, so I'm building 2 separate dfs
columns = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8']
columns2 = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'col11', 'col12']

# where does the PDF live?
pdf_file = 'MO2016.pdf'

# create an empty dataframe
df = pd.DataFrame(columns=columns)
df2 = pd.DataFrame(columns=columns2)

In [254]:
# define function for first page
def page_to_df(page):
    
    # find the table on the page and extract the data
    table = page.extract_table(table_settings={"vertical_strategy": "text"})
    
    # grab all rows in the table 
    # including first one, because our table_settings mean it already ignores header row
    lines = table
    
    # return the data in a data frame
    return pd.DataFrame(lines, columns=columns)

In [255]:
# define function for second page
def page_to_df2(page):
    
    # find the table on the page and extract the data
    table = page.extract_table(table_settings={"vertical_strategy": "text"})
    
    # grab all rows in the table 
    # including first one, because our table_settings mean it already ignores header row
    lines = table
    
    # return the data in a data frame
    return pd.DataFrame(lines, columns=columns2)

In [256]:
# extract the first page
with pdfplumber.open(pdf_file) as pdf:
    pg1 = pdf.pages[0]
    df1 = page_to_df(pg1)

In [257]:
# check out results of first page
df1.head()

,col1,col2,col3,col4,col5,col6,col7,col8
0,Adair,18.2%,"4 ,730",19.0%,900,71%,,29%
1,Andrew,10.9%,"3 ,992",14.1%,560,73%,,27%
2,Atchison,12.9%,"1 ,041",17.4%,180,67%,,33%
3,Audrain,14.9%,"6 ,013",19.9%,"1,190",73%,,27%
4,Barry,13.9%,"8 ,181",21.6%,"1,770",88%,,12%


In [258]:
# extract the second page
with pdfplumber.open(pdf_file) as pdf:
    pg2 = pdf.pages[1]
    df2 = page_to_df2(pg2)

In [259]:
# check out results of second page
df2.head()

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12
0,Madison,,14.5%,"2 ,880",,18.7%,,540,100%,,0%,
1,Maries,,14.4%,"1 ,921",,22.4%,,430,87%,,13%,
2,Marion,,15.2%,"6 ,825",,19.2%,,"1,310",74%,,26%,
3,Mercer,,13.2%,9 26,,18.0%,,170,91%,,10%,
4,Miller,,14.6%,"5 ,899",,21.1%,,"1,240",74%,,26%,


In [260]:
# remove empty columns in each df
# df1
df1 = df1.drop(['col7'], axis=1)
df1.head()

,col1,col2,col3,col4,col5,col6,col8
0,Adair,18.2%,"4 ,730",19.0%,900,71%,29%
1,Andrew,10.9%,"3 ,992",14.1%,560,73%,27%
2,Atchison,12.9%,"1 ,041",17.4%,180,67%,33%
3,Audrain,14.9%,"6 ,013",19.9%,"1,190",73%,27%
4,Barry,13.9%,"8 ,181",21.6%,"1,770",88%,12%


In [261]:
# change col names so we can merge
df1 = df1.rename(index=str, columns={"col1": "county", "col2": "insecure_rate", "col3": "child_pop", 
                                    "col4": "child_insecure_rate", "col5": "child_insecure_pop", 
                                     "col6": "insecure_eligible", "col8": "insecure_ineligible"})
df1.head()

,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
0,Adair,18.2%,"4 ,730",19.0%,900,71%,29%
1,Andrew,10.9%,"3 ,992",14.1%,560,73%,27%
2,Atchison,12.9%,"1 ,041",17.4%,180,67%,33%
3,Audrain,14.9%,"6 ,013",19.9%,"1,190",73%,27%
4,Barry,13.9%,"8 ,181",21.6%,"1,770",88%,12%


In [262]:
# df2
df2 = df2.drop(['col2', 'col5', 'col7', 'col10', 'col12'], axis=1)
df2.head()

,col1,col3,col4,col6,col8,col9,col11
0,Madison,14.5%,"2 ,880",18.7%,540,100%,0%
1,Maries,14.4%,"1 ,921",22.4%,430,87%,13%
2,Marion,15.2%,"6 ,825",19.2%,"1,310",74%,26%
3,Mercer,13.2%,9 26,18.0%,170,91%,10%
4,Miller,14.6%,"5 ,899",21.1%,"1,240",74%,26%


In [263]:
# change col names so we can merge
df2 = df2.rename(index=str, columns={"col1": "county", "col3": "insecure_rate", "col4": "child_pop", 
                                    "col6": "child_insecure_rate", "col8": "child_insecure_pop", 
                                     "col9": "insecure_eligible", "col11": "insecure_ineligible"})
df2.head()

,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
0,Madison,14.5%,"2 ,880",18.7%,540,100%,0%
1,Maries,14.4%,"1 ,921",22.4%,430,87%,13%
2,Marion,15.2%,"6 ,825",19.2%,"1,310",74%,26%
3,Mercer,13.2%,9 26,18.0%,170,91%,10%
4,Miller,14.6%,"5 ,899",21.1%,"1,240",74%,26%


In [264]:
data = pd.concat([df1,df2])
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116 entries, 0 to 54
Data columns (total 7 columns):
county                 116 non-null object
insecure_rate          116 non-null object
child_pop              116 non-null object
child_insecure_rate    116 non-null object
child_insecure_pop     116 non-null object
insecure_eligible      116 non-null object
insecure_ineligible    116 non-null object
dtypes: object(7)
memory usage: 7.2+ KB


In [265]:
# DATA CLEANING

# strip middle whitespace
data['child_pop'] = data['child_pop'].str.strip().str.lower().str.replace(' ', '')

# and trim any other leading/trailing spaces

# remove percent signs
data['insecure_rate'] = data['insecure_rate'].str.strip().str.lower().str.replace('%', '')
data['child_insecure_rate'] = data['child_insecure_rate'].str.strip().str.lower().str.replace('%', '')
data['insecure_eligible'] = data['insecure_eligible'].str.strip().str.lower().str.replace('%', '')
data['insecure_ineligible'] = data['insecure_ineligible'].str.strip().str.lower().str.replace('%', '')


In [266]:
# turn percents into numbers
data['insecure_rate'] = pd.to_numeric(data['insecure_rate'], errors='coerce')
data['child_insecure_rate'] = pd.to_numeric(data['child_insecure_rate'], errors='coerce')
data['insecure_eligible'] = pd.to_numeric(data['insecure_eligible'], errors='coerce')
data['insecure_ineligible'] = pd.to_numeric(data['insecure_ineligible'], errors='coerce')

# turn percents into decimals
data['insecure_rate'] = (data['insecure_rate'] / 100)
data['child_insecure_rate'] = (data['child_insecure_rate'] / 100)
data['insecure_eligible'] = (data['insecure_eligible'] / 100)
data['insecure_ineligible'] = (data['insecure_ineligible'] / 100)

In [267]:
data.head()

,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
0,Adair,0.182,"4,730",0.190,900,0.71,0.29
1,Andrew,0.109,"3,992",0.141,560,0.73,0.27
2,Atchison,0.129,"1,041",0.174,180,0.67,0.33
3,Audrain,0.149,"6,013",0.199,"1,190",0.73,0.27
4,Barry,0.139,"8,181",0.216,"1,770",0.88,0.12


In [268]:
# reset index
data = data.reset_index()

In [271]:
# remove last row containing summary data
data = data.drop(115)

In [272]:
# check end of file
data.tail()

,index,county,insecure_rate,child_pop,child_insecure_rate,child_insecure_pop,insecure_eligible,insecure_ineligible
110,49,Wayne,0.157,"2,805",0.224,630,0.88,0.12
111,50,Webster,0.145,"10,091",0.201,"2,020",0.90,0.10
112,51,Worth,0.118,415,0.168,70,1.00,0.00
113,52,Wright,0.173,"4,587",0.233,"1,070",0.93,0.07
114,53,St. Louis city,0.251,"63,972",0.233,"14,890",0.60,0.40


In [276]:
# write to csv
data.to_csv('foodinsecurity_MO.csv')